In [1]:
"""
Created on Thu Apr 25 20:22:30 2020
@author: Kartik
"""

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from datetime import datetime
import geocoder
import cv2
from mtcnn.mtcnn import MTCNN
import numpy as np
from keras.preprocessing.image import img_to_array
from resizeimage import resizeimage
from keras.models import load_model
from keras.preprocessing.image import load_img


Using TensorFlow backend.


In [6]:
# Initialising the CNN
classifier = Sequential() #object of class sequential.

# Step 1 - Convolution
classifier.add(Convolution2D(32, (3, 3), input_shape = (160, 160, 3), activation = 'relu'))
#(32-no of filters/feature map, no of rows & col of feature detector matrix,I_shape=3-no of channel
#colour image, 64*64 format for image converstion, )

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# to reduce no. of nodes in flattening, 2,2 to size the mat that slides over.
# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(activation="relu", units=128)) #rectifier A FN for input/hidden layer.
classifier.add(Dense(activation="sigmoid", units=1)) #sigmoid layer.

#classifier.add(Dense())
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('C:/Users/KPATNAIk/Desktop/mask/dataset/training_set',
                                                 target_size = (160, 160),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('C:/Users/KPATNAIk/Desktop/mask/dataset/test_set',
                                            target_size = (160, 160),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 250,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 32)


classifier.save('C:/Users/KPATNAIk/Desktop/mask/mask.h5')

C:\Users\KPATNAIk\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  del sys.path[0]


Found 2206 images belonging to 2 classes.
Found 553 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 274s 1s/step - loss: 0.3522 - accuracy: 0.8771 - val_loss: 0.0949 - val_accuracy: 0.9580
Epoch 2/25
250/250 [==============================] - 257s 1s/step - loss: 0.1112 - accuracy: 0.9617 - val_loss: 0.1144 - val_accuracy: 0.9530
Epoch 3/25
250/250 [==============================] - 243s 971ms/step - loss: 0.0816 - accuracy: 0.9715 - val_loss: 0.1029 - val_accuracy: 0.9611
Epoch 4/25
250/250 [==============================] - 229s 918ms/step - loss: 0.0616 - accuracy: 0.9787 - val_loss: 0.1030 - val_accuracy: 0.9703
Epoch 5/25
250/250 [==============================] - 260s 1s/step - loss: 0.0550 - accuracy: 0.9816 - val_loss: 0.0594 - val_accuracy: 0.9740
Epoch 6/25
250/250 [==============================] - 286s 1s/step - loss: 0.0563 - accuracy: 0.9825 - val_loss: 0.0647 - val_accuracy: 0.9622
Epoch 7/25
250/250 [==============================] -

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 23 22:00:03 2020
@author: Abhishek
"""

g = geocoder.ip('me')
a=str(g.latlng[0])
b=str(g.latlng[1])
c='GPS::'+a+' '+b

detector = MTCNN()

model = load_model("C:/Users/KPATNAIk/Desktop/mask/mask.h5")


font = cv2.FONT_HERSHEY_TRIPLEX
font2 = cv2.FONT_HERSHEY_COMPLEX_SMALL
font3 = cv2.FONT_HERSHEY_SCRIPT_COMPLEX
font4 = cv2.FONT_HERSHEY_SIMPLEX
#image_file = load_img('test_img.jpg')

#print(chr(169))
#rights=chr(169)+'2020'

cap = cv2.VideoCapture(0)
while True: 
    
    #Capture frame-by-frame
    __, frame = cap.read()
    
    cv2.putText(frame,str(datetime.now()),(10,30), font3, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,c,(10,450), font2, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,'Kartik\'s CAM',(480,450), font2, 0.9,(255,255,255),2,cv2.LINE_AA)
    #Use MTCNN to detect faces
    result = detector.detect_faces(frame)
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
            #cv2.putText(frame,"The Face",(200,100), font, 1,(255,255,255),2,cv2.LINE_AA)
            cv2.imwrite('opencv.png', frame)
            image_file = load_img('opencv.png')
            cover = resizeimage.resize_cover(image_file, [160, 160], validate=False)


            x = []
            
            
            x = img_to_array(cover)
            
            x = np.expand_dims(x, axis=0)
            
            pred = model.predict(x)
            
            #print(pred[0][0])
            
            if pred[0][0]==0.0:
                cv2.putText(frame,"ACCESS GRANTED, MASK ON",(100,100), font4, 0.8,(0,255,0),2,cv2.LINE_AA)
            
            elif pred[0][0]==1.0:
                cv2.putText(frame,"ACCESS DENIED, No FACE-MASK",(100,100), font4, 0.8,(0,0,255),2,cv2.LINE_AA)
            
            cv2.rectangle(frame,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,155,255),
                          2)
    
            cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
    #display resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(10) &0xFF == ord('q'):
        break
#When everything's done, release capture
cap.release()
cv2.destroyAllWindows()